### Important Note
This notebook was based on DE 2025 Lab 4

### Installation
Install the packages required for executing this notebook.

In [1]:
# Install the packages
! pip3 install --user --no-cache-dir --upgrade "kfp>2" "google-cloud-pipeline-components>2" \
                                        google-cloud-aiplatform

## Restart the kernel
Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [2]:
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Check the versions of the packages you installed. The KFP SDK version should be >2.

In [2]:
! python3 -c "import kfp; print('KFP SDK version: {}'.format(kfp.__version__))"
! pip3 freeze | grep aiplatform
! python3 -c "import google_cloud_pipeline_components; print('google_cloud_pipeline_components version: {}'.format(google_cloud_pipeline_components.__version__))"

KFP SDK version: 2.14.6
google-cloud-aiplatform==1.121.0
google_cloud_pipeline_components version: 2.21.0


In [3]:
import kfp
import typing
from typing import Dict
from typing import NamedTuple
from kfp import dsl
from kfp.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        ClassificationMetrics,
                        component, 
                        OutputPath, 
                        InputPath)
import google.cloud.aiplatform as aip
from google_cloud_pipeline_components.types import artifact_types

#### Project and Pipeline Configurations

In [97]:
#The Google Cloud project that this pipeline runs in.
PROJECT_ID = "de2025-group10"
# The region that this pipeline runs in
REGION = "europe-west4"
# Specify a Cloud Storage URI that your pipelines service account can access. The artifacts of your pipeline runs are stored within the pipeline root.
PIPELINE_ROOT = "gs://de2025-group10-a1-pipelines"

#### Function to preprocess the dataset

#### Pipeline Component : Data Ingestion

In [109]:
@dsl.component(
    packages_to_install=["pandas","google-cloud-storage"],
    base_image="python:3.10.7-slim"
)
def download_data(project_id: str, bucket: str, file_name: str, dataset: Output[Dataset]):
    '''download data'''
    from google.cloud import storage
    import pandas as pd
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    # Downloaing the file from a google bucket 
    client = storage.Client(project=project_id)
    bucket = client.bucket(bucket)
    blob = bucket.blob(file_name)
    blob.download_to_filename(dataset.path + ".csv")
    logging.info(f'Downloaded Data {dataset.path}!')

#### Pipeline Component : Training-LR (LogisticRegression)

In [128]:
@dsl.component(
    packages_to_install=['pandas', 'scikit-learn==1.3.2'],
    base_image="python:3.10.7-slim"
)
def train_lr(train_data: Input[Dataset], test_data: Input[Dataset], out_model: Output[Model]) -> NamedTuple('outputs', metrics=dict):
    import pandas as pd
    from sklearn.linear_model import LogisticRegression
    from sklearn.metrics import classification_report, accuracy_score
    import pickle
    import logging 
    import sys
    import os
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df_train = pd.read_csv(train_data.path+".csv")
    logging.info("Train Data Glimpse:")
    logging.info(df_train.head())
    X_train = df_train.iloc[:, :-1]
    Y_train = df_train["Survived"]
    
    df_test = pd.read_csv(test_data.path+".csv")
    logging.info("Test Data Glimpse:")
    logging.info(df_test.head())
    X_test = df_test.iloc[:, :-1]
    Y_test = df_test["Survived"]
    
    # TODO: Load Test data to assess model;
    # Save metrics from test data assessment
    
    lr_model = LogisticRegression()
    lr_model.fit(X_train, Y_train)
    
    predicted_on_trainset = lr_model.predict(X_train)
    logging.info("Testing model results using test data...")
    logging.info(classification_report(Y_train, predicted_on_trainset))
    
    y_pred = lr_model.predict(X_test)
    metrics_dict = {
        "accuracy": accuracy_score(Y_test, y_pred)
    }
    
    out_model.metadata["file_type"] = ".pkl"
    out_model.metadata["algo"] = "lr"
    
    m_file = out_model.path + ".pkl"
    with open(m_file, "wb") as f:
        pickle.dump(lr_model, f)
        
    outputs = NamedTuple('outputs', metrics=dict)
    return outputs(metrics_dict)

#### Pipeline Component : Training-RF (RandomForest)

In [129]:
@dsl.component(
    packages_to_install=['pandas', 'scikit-learn==1.3.2'],
    base_image="python:3.10.7-slim"
)
def train_rf(train_data: Input[Dataset], test_data: Input[Dataset], out_model: Output[Model]) -> NamedTuple('outputs', metrics=dict):
    import pandas as pd
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.metrics import classification_report, accuracy_score
    import pickle
    import logging 
    import sys
    import os
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df_train = pd.read_csv(train_data.path+".csv")
    logging.info("Train Data Glimpse:")
    logging.info(df_train.head())
    X_train = df_train.iloc[:, :-1]
    Y_train = df_train["Survived"]
    
    df_test = pd.read_csv(test_data.path+".csv")
    logging.info("Test Data Glimpse:")
    logging.info(df_test.head())
    X_test = df_test.iloc[:, :-1]
    Y_test = df_test["Survived"]
    
    rf_model = RandomForestClassifier()
    rf_model.fit(X_train, Y_train)
    
    predicted_on_trainset = rf_model.predict(X_train)
    logging.info("Testing model results using test data...")
    logging.info(classification_report(Y_train, predicted_on_trainset))
    
    y_pred = rf_model.predict(X_test)
    metrics_dict = {
        "accuracy": accuracy_score(Y_test, y_pred)
    }
    
    out_model.metadata["file_type"] = ".pkl"
    out_model.metadata["algo"] = "lr"
    
    m_file = out_model.path + ".pkl"
    with open(m_file, "wb") as f:
        pickle.dump(rf_model, f)
        
    outputs = NamedTuple('outputs', metrics=dict)
    return outputs(metrics_dict)

#### Pipeline Component : Prediction-LR (LogisticRegression)

In [138]:
@dsl.component(
    packages_to_install=['pandas', 'scikit-learn==1.3.2'],
    base_image="python:3.10.7-slim"
)
def predict_lr(model: Input[Model], test_data: Input[Dataset], results: Output[Dataset]):
    import pandas as pd
    import pickle  
    import json
    import logging
    import sys
    import os
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df_test = pd.read_csv(test_data.path+".csv")
       
    filename = model.path + ".pkl"
        
    #Loading the saved model
    model_lr = pickle.load(open(filename, 'rb'))

    X_test = df_test.iloc[:, :-1]
    Y_test = df_test["Survived"]

    dfcp = df_test.copy()   
    y_pred = model_lr.predict(X_test)
    logging.info(y_pred)
    dfcp['y_pred'] = y_pred.tolist()     
    dfcp.to_csv(results.path + ".csv" , index=False, encoding='utf-8-sig')

#### Pipeline Component : Prediction-RF (RandomForest)

In [139]:
@dsl.component(
    packages_to_install=['pandas', 'scikit-learn==1.3.2'],
    base_image="python:3.10.7-slim"
)
def predict_rf(model: Input[Model], test_data: Input[Dataset], results: Output[Dataset]):
    import pandas as pd
    import pickle  
    import json
    import logging
    import sys
    import os
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    
    df_test = pd.read_csv(test_data.path+".csv")
       
    filename = model.path + ".pkl"
        
    #Loading the saved model
    model_rf = pickle.load(open(filename, 'rb'))

    X_test = df_test.iloc[:, :-1]
    Y_test = df_test["Survived"]

    dfcp = df_test.copy()   
    y_pred = model_rf.predict(X_test)
    logging.info(y_pred)
    dfcp['y_pred'] = y_pred.tolist()     
    dfcp.to_csv(results.path + ".csv" , index=False, encoding='utf-8-sig')

#### Pipeline Component : Algorithm Selection 

In [140]:
@dsl.component(
    base_image="python:3.10.7-slim"
)
def compare_model(lr_metrics: dict, rf_metrics: dict) -> str:
    import logging
    import json
    import sys
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)
    logging.info(f"Logistic Regression Metrics: {lr_metrics}")
    logging.info(f"Random Forest Metrics: {rf_metrics}")
    if lr_metrics.get("accuracy") > rf_metrics.get("accuracy"):
        return "LR"
    else :
        return "RF"

### Upload Model and Metrics to Google Bucket 

In [141]:
@dsl.component(
    packages_to_install=["google-cloud-storage"],
    base_image="python:3.10.7-slim"
)
def upload_model_to_gcs(project_id: str, model_repo: str, model: Input[Model]):
    '''upload model to gsc'''
    from google.cloud import storage   
    import logging 
    import sys
    
    logging.basicConfig(stream=sys.stdout, level=logging.INFO)    
  
    # upload the model to GCS
    client = storage.Client(project=project_id)
    bucket = client.bucket(model_repo)
    #blob = bucket.blob(str(model.metadata["algo"]) + '_model' + str(model.metadata["file_type"]))
    blob = bucket.blob("best_model" + str(model.metadata["file_type"]))
    blob.upload_from_filename(model.path + str(model.metadata["file_type"]))       
    
    print("Saved the model to GCP bucket : " + model_repo)

#### Define the Pipeline

In [142]:
# Define the workflow of the pipeline.
@kfp.dsl.pipeline(
    name="titanic-predictor-training-pipeline")
def pipeline(project_id: str, data_bucket: str, trainset_filename: str, model_repo: str, testset_filename: str):
    
    
    load_train_data_op = download_data(
        project_id=project_id,
        bucket=data_bucket,
        file_name=trainset_filename
    )
    
    load_test_data_op = download_data(
        project_id=project_id,
        bucket=data_bucket,
        file_name=testset_filename
    )

 
    training_lr_job_run_op = train_lr(
        train_data = load_train_data_op.outputs["dataset"],
        test_data = load_test_data_op.outputs["dataset"]
    )
    
     
    training_rf_job_run_op = train_rf(
        train_data = load_train_data_op.outputs["dataset"],
        test_data = load_test_data_op.outputs["dataset"]
    )
        
    comp_model_op = compare_model(
        lr_metrics=training_lr_job_run_op.outputs["metrics"],                
        rf_metrics=training_rf_job_run_op.outputs["metrics"]
    ).after(training_lr_job_run_op, training_rf_job_run_op)  
    
    # defining the branching condition
    with dsl.If(comp_model_op.output=="LR"):
        predict_lr_job_run_op = predict_lr(
            model=training_lr_job_run_op.outputs["out_model"],      
            test_data=load_test_data_op.outputs["dataset"]
        )
        upload_model_lr_to_gc_op = upload_model_to_gcs(
            project_id=project_id,
            model_repo=model_repo,
            model=training_lr_job_run_op.outputs['out_model']
        ).after(predict_lr_job_run_op)
        
    with dsl.If(comp_model_op.output=="RF"):
        predict_rf_job_run_op = predict_rf(
            model=training_rf_job_run_op.outputs["out_model"],      
            test_data=load_test_data_op.outputs["dataset"]
        )
        upload_model_rf_to_gc_op = upload_model_to_gcs(
            project_id=project_id,
            model_repo=model_repo,
            model=training_rf_job_run_op.outputs['out_model']
        ).after(predict_rf_job_run_op)

#### Compile the pipeline into a JSON file

In [143]:
from kfp import compiler
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='titanic_predictor_training_pipeline.yaml')

#### Submit the pipeline run

In [144]:
import google.cloud.aiplatform as aip

# Before initializing, make sure to set the GOOGLE_APPLICATION_CREDENTIALS
# environment variable to the path of your service account.
aip.init(
    project=PROJECT_ID,
    staging_bucket=PIPELINE_ROOT,
)

# Prepare the pipeline job
job = aip.PipelineJob(
    display_name="titanic-predictor-ct-pipeline",
    enable_caching=False,
    template_path="titanic_predictor_training_pipeline.yaml",
    pipeline_root=PIPELINE_ROOT,
    location=REGION,
    parameter_values={
        'project_id': PROJECT_ID, # project id 
        'data_bucket': 'de2025-group10-a1-data',  # data bucket 
        'trainset_filename': 'titanic/train.csv', 
        'testset_filename': 'titanic/test.csv',    
        'model_repo':'de2025-group10-a1-models' # model bucket
    }
)

job.run()